## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,26.98,68,68,5.59,broken clouds
1,1,Natal,BR,-5.7950,-35.2094,86.65,66,40,14.97,scattered clouds
2,2,Bambous Virieux,MU,-20.3428,57.7575,77.25,78,40,11.50,light rain
3,3,Quelimane,MZ,-17.8786,36.8883,73.80,93,9,5.88,light rain
4,4,Airai,TL,-8.9266,125.4092,59.04,88,71,0.74,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             225
City                   225
Country                221
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

Unnamed: 0             221
City                   221
Country                221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df =preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Natal,BR,86.65,scattered clouds,-5.7950,-35.2094,
2,Bambous Virieux,MU,77.25,light rain,-20.3428,57.7575,
9,Souillac,MU,77.38,light rain,-20.5167,57.5167,
18,Galle,LK,80.15,scattered clouds,6.0367,80.2170,
20,Kodinar,IN,83.28,clear sky,20.7903,70.7031,
22,Barabai,ID,76.26,moderate rain,-2.5833,115.3833,
26,Kampong Thum,KH,78.51,overcast clouds,12.7111,104.8887,
28,Arraial Do Cabo,BR,82.36,few clouds,-22.9661,-42.0278,
30,Cabo San Lucas,MX,83.61,clear sky,22.8909,-109.9124,
35,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Natal,BR,86.65,scattered clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
2,Bambous Virieux,MU,77.25,light rain,-20.3428,57.7575,Casa Tia Villa
9,Souillac,MU,77.38,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
18,Galle,LK,80.15,scattered clouds,6.0367,80.2170,Amangalla
20,Kodinar,IN,83.28,clear sky,20.7903,70.7031,residency of N.B vadhel
22,Barabai,ID,76.26,moderate rain,-2.5833,115.3833,HOTEL BOURAQ INDAH PERMAI
26,Kampong Thum,KH,78.51,overcast clouds,12.7111,104.8887,Sambor Village Hotel
28,Arraial Do Cabo,BR,82.36,few clouds,-22.9661,-42.0278,Pousada Porto Praia
30,Cabo San Lucas,MX,83.61,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
35,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()



In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))